# 일반 테스트 -> 허깅페이스 한국어뉴스 데이터셋
 - ROUGE - 1
 - ROUGE - 2
 - ROUGE - L

In [1]:
#필요한 라이브러리 설치
!pip install transformers torch rouge-score konlpy keybert scikit-learn PyMuPDF nltk
!pip install sentence-transformers

# Java 설치 (Komoran 사용을 위해 필요)
!apt-get update
!apt-get install -y openjdk-8-jdk
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
!pip install -U datasets huggingface_hub fsspec

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.3 MB/s eta 0:00:00
   ━━

## 기본 테스트

In [2]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from datasets import load_dataset
import numpy as np
from rouge_score import rouge_scorer
import re
from tqdm import tqdm
import random

In [3]:
# 모델과 토크나이저 로드
def load_kobart_model():
    """KoBART 모델과 토크나이저 로드"""
    model_name = 'digit82/kobart-summarization'

    print("모델과 토크나이저 로딩 중...")
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # GPU 사용 가능하면 GPU로 이동
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    print(f"모델이 {device}에 로드되었습니다.")

    return model, tokenizer, device

In [11]:
# 단일 텍스트 요약 함수
def summarize_text(text, model, tokenizer, device, max_input_length=1024):
    """단일 텍스트를 요약하는 함수"""
    # 입력 텍스트 토크나이징
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length
    ).to(device)

    # 요약 생성
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids,
            num_beams=6,
            max_length=150,
            min_length=80,
            repetition_penalty=3.0,
            length_penalty=0.8,
            early_stopping=True,
            no_repeat_ngram_size=4
        )

    # 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [5]:
# 배치 평가 함수
def evaluate_model_performance(model, tokenizer, device, test_data, num_samples=100):
    """모델 성능을 평가하는 함수"""
    print(f"\n{num_samples}개 샘플로 성능 평가를 시작합니다...")

    # 랜덤 샘플링
    if len(test_data) > num_samples:
        indices = random.sample(range(len(test_data)), num_samples)
        test_samples = [test_data[i] for i in indices]
    else:
        test_samples = test_data

    predictions = []
    references = []

    # 각 샘플에 대해 요약 생성
    for i, sample in enumerate(tqdm(test_samples, desc="요약 생성 중")):
        try:
            # 원본 텍스트와 참조 요약
            passage = sample['passage']
            reference = sample['summary']

            # 모델로 요약 생성
            prediction = summarize_text(passage, model, tokenizer, device)

            predictions.append(prediction)
            references.append(reference)

            # 진행상황 출력 (10개마다)
            if (i + 1) % 10 == 0:
                print(f"\n--- 샘플 {i+1} ---")
                print(f"원본 길이: {len(passage)} 문자")
                print(f"참조 요약: {reference[:100]}...")
                print(f"생성 요약: {prediction[:100]}...")

        except Exception as e:
            print(f"샘플 {i} 처리 중 오류: {e}")
            continue

    return predictions, references

In [6]:
# ROUGE 점수 계산 함수
def evaluate_rouge_korean(predictions, references):
    """한국어 텍스트를 위한 ROUGE 점수 계산"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    print("\nROUGE 점수 계산 중...")
    for pred, ref in tqdm(zip(predictions, references), total=len(predictions)):
        # 한국어 텍스트 전처리
        pred_clean = re.sub(r'\s+', ' ', pred.strip())
        ref_clean = re.sub(r'\s+', ' ', ref.strip())

        scores = scorer.score(ref_clean, pred_clean)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': {
            'mean': np.mean(rouge1_scores),
            'std': np.std(rouge1_scores)
        },
        'rouge2': {
            'mean': np.mean(rouge2_scores),
            'std': np.std(rouge2_scores)
        },
        'rougeL': {
            'mean': np.mean(rougeL_scores),
            'std': np.std(rougeL_scores)
        }
    }

In [7]:
# 결과 출력 함수
def print_evaluation_results(rouge_scores):
    """평가 결과를 보기 좋게 출력"""
    print("\n" + "="*50)
    print("KoBART 모델 성능 평가 결과")
    print("="*50)

    for metric, scores in rouge_scores.items():
        print(f"{metric.upper():>8}: {scores['mean']:.4f} (±{scores['std']:.4f})")

    print("="*50)
    print("* 점수가 높을수록 좋은 성능을 의미합니다.")
    print("* ROUGE-1: 단어 단위 겹침")
    print("* ROUGE-2: 2-gram 단위 겹침")
    print("* ROUGE-L: 최장 공통 부분수열")

In [9]:
# 메인 실행 함수
def main():
    """메인 실행 함수"""
    # 1. 데이터셋 로드
    print("데이터셋 로딩 중...")
    dataset = load_dataset("Laplace04/KoreanSummarizeAiHub")
    test_data = dataset['test']
    print(f"테스트 데이터 크기: {len(test_data)}")

    # 2. 모델 로드
    model, tokenizer, device = load_kobart_model()

    # 3. 성능 평가 (100개 샘플로 테스트, 필요시 조정 가능)
    predictions, references = evaluate_model_performance(
        model, tokenizer, device, test_data, num_samples=100
    )

    # 4. ROUGE 점수 계산
    rouge_scores = evaluate_rouge_korean(predictions, references)

    # 5. 결과 출력
    print_evaluation_results(rouge_scores)

    # 6. 샘플 결과 출력
    print("\n" + "="*50)
    print("샘플 요약 결과 (처음 3개)")
    print("="*50)

    for i in range(min(3, len(predictions))):
        print(f"\n--- 샘플 {i+1} ---")
        print(f"참조 요약: {references[i]}")
        print(f"생성 요약: {predictions[i]}")
        print("-" * 30)

In [13]:
main()

데이터셋 로딩 중...
테스트 데이터 크기: 9150
모델과 토크나이저 로딩 중...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


모델이 cuda에 로드되었습니다.

100개 샘플로 성능 평가를 시작합니다...


요약 생성 중:  10%|█         | 10/100 [00:09<01:33,  1.03s/it]


--- 샘플 10 ---
원본 길이: 757 문자
참조 요약: 정부는 지난 6월 집단 식중독이 발생한 안산 A유치원에 대한 역학조사 진행 후, 역학조사 방해 혐의로 A유치원을 고발하고, 유치원 및 어린이집 전수점검(7.6~31)을 실시한 후 ...
생성 요약: 정부는 지난 6월 집단 식중독이 발생한 안산 A유치원에 대한 역학조사 진행 후, 유치원 및 어린이집 전수점검(7.6~31)을 실시한 후 점검 결과를 분석하여 관계부처 합동으로 급식...


...


--- 샘플 90 ---
원본 길이: 1235 문자
참조 요약: 정부가 기부금 사용 논란이 불거진 위안부 피해자 지원단체 정의기억연대(정의연)에 대해서 위법사항이 발견되면 조치를 취하겠다는 입장을 밝혔다. 행안위 소속 의원들은 이날 회의에서 정...
생성 요약: 정부가 19일금 사용 논란이 불거진 위안부 피해자 지원단체 정의기억연대(정의연)에 대해서 위법사항이 발견되면 조치를 취하겠다는 입장을 밝히자 진영 행정안전부 장관은 19일 국회에서...


요약 생성 중: 100%|██████████| 100/100 [01:27<00:00,  1.14it/s]



--- 샘플 100 ---
원본 길이: 870 문자
참조 요약: 굉장히 어려운 질문을 해 주셨는데, 우선 첫 번째 질문과 관련해서는 문자와 언어를 우선 분명하게 구분할 필요가 있습니다. 그러니까 한글은 우리 국어를 표기하는 문자이고, 한국어는 ...
생성 요약: 한글은 우리 국어를 표기하는 문자이고, 한국어는 언어인데, 많은 분들이 이 한글을 한국어, 즉 언어라고 오해하는 부분들이 있어서, 여러분은 그렇지 않으시겠으나 문자가 없는 민족에게...

ROUGE 점수 계산 중...


100%|██████████| 100/100 [00:00<00:00, 10372.44it/s]


KoBART 모델 성능 평가 결과
  ROUGE1: 0.3173 (±0.3730)
  ROUGE2: 0.2177 (±0.3407)
  ROUGEL: 0.3148 (±0.3716)
* 점수가 높을수록 좋은 성능을 의미합니다.
* ROUGE-1: 단어 단위 겹침
* ROUGE-2: 2-gram 단위 겹침
* ROUGE-L: 최장 공통 부분수열

샘플 요약 결과 (처음 3개)

--- 샘플 1 ---
참조 요약: 그러나 간호원은 핼끔 한 번 박의사를 돌아다 보기만 했을 뿐, 대답이 없다. 콘로 위에다 주사기를 간호원은 끓이고 서 있었다. 박의사는 처방지에 미그로낭을 섞은 약제를 써 가지고 "간호원!" 하고 또 한 번 불렀다.
생성 요약: 박의사는 처방지에 미그로낭을 섞은 약제를 써 가지고 "간호원!" 하고 또 한 번 불렀지만 간호원은 보굴 보굴 끓는 알마이트 그릇만 열심히 들여다 보고 서 있는 상태였으며 엔간한 호인인 박의사도 언성을 다소 높였지만  대답하기가 싫어서 안했다며 조용한 말투로 대답해 주었다.
------------------------------

--- 샘플 2 ---
참조 요약: 국가보훈처는 내일 오전 9시 40분에 1급 중상이 국가유공자 22명으로 구성된 나라사랑 국토종단 희망의 핸드사이클 팀이 기탁한 성금 2,000만원을 에티오피아 생존 참전용사에게 전달합니다. 전달되는 성금은 나라사랑 국토종단 희망의 핸드사이클 팀의 지난 10월 15일부터 23일까지 700km 국토 종단 행사를 통해 조성한 성금입니다.
생성 요약: 병무청은 12월 19일 제18대 대통령선거일 주소지에서 투표할 수 없는 군 입영 대상자에 대하여 내일부터 11월 25일까지 주민등록지에 시·군·구청 또는 읍·면·동사무소 등 주민센터에 부재자 신고를 할 수 있도록 입영통지서와 휴대전화 문자서비스를 하고 있으며, 국가보훈처는 내일 오전 9시 40분에 1급 중상이 국가유공자 22명으로 구성된 나라사랑 국토종단 희망의 핸드사이클 팀이 기탁한 성금 2,000만원을 에티오피아 생존 참전용사

# 교차검증

In [14]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from datasets import load_dataset
import numpy as np
from rouge_score import rouge_scorer
import re
from tqdm import tqdm
import random

In [15]:
# 개선된 요약 함수 - 여러 파라미터 세트 테스트
def summarize_text_tuned(text, model, tokenizer, device, config_name="default"):
    """다양한 파라미터 설정으로 텍스트 요약"""

    # 여러 파라미터 설정
    configs = {
        "default": {
            "num_beams": 6,
            "max_length": 150,
            "min_length": 80,
            "repetition_penalty": 3.0,
            "length_penalty": 0.8,
            "no_repeat_ngram_size": 4
        },
        "concise": {  # 더 간결한 요약을 위한 설정
            "num_beams": 4,
            "max_length": 120,
            "min_length": 50,
            "repetition_penalty": 2.5,
            "length_penalty": 1.2,  # 길이 패널티 증가
            "no_repeat_ngram_size": 3
        },
        "focused": {  # 핵심 내용에 집중
            "num_beams": 5,
            "max_length": 100,
            "min_length": 40,
            "repetition_penalty": 2.8,
            "length_penalty": 1.5,  # 더 강한 길이 패널티
            "no_repeat_ngram_size": 3
        },
        "balanced": {  # 균형잡힌 설정
            "num_beams": 5,
            "max_length": 130,
            "min_length": 60,
            "repetition_penalty": 2.7,
            "length_penalty": 1.0,
            "no_repeat_ngram_size": 3
        }
    }

    config = configs[config_name]

    # 입력 텍스트 토크나이징
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    # 요약 생성
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids,
            num_beams=config["num_beams"],
            max_length=config["max_length"],
            min_length=config["min_length"],
            repetition_penalty=config["repetition_penalty"],
            length_penalty=config["length_penalty"],
            early_stopping=True,
            no_repeat_ngram_size=config["no_repeat_ngram_size"]
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [16]:
# 여러 설정으로 평가하는 함수
def evaluate_multiple_configs(model, tokenizer, device, test_data, num_samples=50):
    """여러 파라미터 설정으로 성능 비교"""

    configs = ["default", "concise", "focused", "balanced"]
    results = {}

    # 랜덤 샘플링
    if len(test_data) > num_samples:
        indices = random.sample(range(len(test_data)), num_samples)
        test_samples = [test_data[i] for i in indices]
    else:
        test_samples = test_data

    references = [sample['summary'] for sample in test_samples]

    for config_name in configs:
        print(f"\n=== {config_name.upper()} 설정으로 평가 중 ===")

        predictions = []

        for sample in tqdm(test_samples, desc=f"{config_name} 요약 생성"):
            try:
                prediction = summarize_text_tuned(
                    sample['passage'], model, tokenizer, device, config_name
                )
                predictions.append(prediction)
            except Exception as e:
                print(f"오류 발생: {e}")
                predictions.append("")

        # ROUGE 점수 계산
        rouge_scores = evaluate_rouge_korean(predictions, references)
        results[config_name] = {
            'rouge_scores': rouge_scores,
            'predictions': predictions
        }

        # 결과 출력
        print(f"\n{config_name.upper()} 설정 결과:")
        for metric, scores in rouge_scores.items():
            print(f"  {metric.upper()}: {scores['mean']:.4f} (±{scores['std']:.4f})")

    return results, references

In [17]:
# ROUGE 점수 계산 함수 (기존과 동일)
def evaluate_rouge_korean(predictions, references):
    """한국어 텍스트를 위한 ROUGE 점수 계산"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for pred, ref in zip(predictions, references):
        pred_clean = re.sub(r'\s+', ' ', pred.strip())
        ref_clean = re.sub(r'\s+', ' ', ref.strip())

        scores = scorer.score(ref_clean, pred_clean)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': {'mean': np.mean(rouge1_scores), 'std': np.std(rouge1_scores)},
        'rouge2': {'mean': np.mean(rouge2_scores), 'std': np.std(rouge2_scores)},
        'rougeL': {'mean': np.mean(rougeL_scores), 'std': np.std(rougeL_scores)}
    }

In [18]:
# 결과 비교 출력
def compare_results(results):
    """여러 설정의 결과를 비교하여 출력"""
    print("\n" + "="*70)
    print("파라미터 설정별 성능 비교")
    print("="*70)

    configs = list(results.keys())
    metrics = ['rouge1', 'rouge2', 'rougeL']

    # 헤더 출력
    print(f"{'Config':<12} {'ROUGE-1':<12} {'ROUGE-2':<12} {'ROUGE-L':<12}")
    print("-" * 70)

    best_scores = {metric: 0 for metric in metrics}
    best_config = {metric: "" for metric in metrics}

    for config in configs:
        rouge_scores = results[config]['rouge_scores']
        r1 = rouge_scores['rouge1']['mean']
        r2 = rouge_scores['rouge2']['mean']
        rL = rouge_scores['rougeL']['mean']

        print(f"{config:<12} {r1:<12.4f} {r2:<12.4f} {rL:<12.4f}")

        # 최고 점수 추적
        if r1 > best_scores['rouge1']:
            best_scores['rouge1'] = r1
            best_config['rouge1'] = config
        if r2 > best_scores['rouge2']:
            best_scores['rouge2'] = r2
            best_config['rouge2'] = config
        if rL > best_scores['rougeL']:
            best_scores['rougeL'] = rL
            best_config['rougeL'] = config

    print("\n" + "="*70)
    print("최고 성능 설정:")
    for metric in metrics:
        print(f"  {metric.upper()}: {best_config[metric]} ({best_scores[metric]:.4f})")

    return best_config

In [19]:
# 샘플 비교 출력
def show_sample_comparison(results, references, sample_idx=0):
    """특정 샘플에 대한 여러 설정 결과 비교"""
    print(f"\n" + "="*70)
    print(f"샘플 {sample_idx + 1} 비교")
    print("="*70)

    print(f"참조 요약: {references[sample_idx]}")
    print("-" * 70)

    for config_name, result in results.items():
        prediction = result['predictions'][sample_idx]
        print(f"{config_name.upper()} 설정:")
        print(f"  {prediction}")
        print(f"  길이: {len(prediction)} 문자")
        print("-" * 70)

In [20]:
# 모델 로드 함수 (기존과 동일)
def load_kobart_model():
    """KoBART 모델과 토크나이저 로드"""
    model_name = 'digit82/kobart-summarization'

    print("모델과 토크나이저 로딩 중...")
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    print(f"모델이 {device}에 로드되었습니다.")

    return model, tokenizer, device


In [22]:
# 메인 실행 함수
def main():
    """메인 실행 함수"""
    # 1. 데이터셋 로드
    print("데이터셋 로딩 중...")
    dataset = load_dataset("Laplace04/KoreanSummarizeAiHub")
    test_data = dataset['test']

    # 2. 모델 로드
    model, tokenizer, device = load_kobart_model()

    # 3. 여러 설정으로 평가 (100개 샘플 동일하게)
    results, references = evaluate_multiple_configs(
        model, tokenizer, device, test_data, num_samples=100
    )

    # 4. 결과 비교
    best_config = compare_results(results)

    # 5. 샘플 비교 (처음 3개)
    for i in range(min(3, len(references))):
        show_sample_comparison(results, references, i)

In [23]:
main()

데이터셋 로딩 중...
모델과 토크나이저 로딩 중...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


모델이 cuda에 로드되었습니다.

=== DEFAULT 설정으로 평가 중 ===


default 요약 생성: 100%|██████████| 100/100 [01:37<00:00,  1.03it/s]



DEFAULT 설정 결과:
  ROUGE1: 0.3131 (±0.3750)
  ROUGE2: 0.1828 (±0.3148)
  ROUGEL: 0.3025 (±0.3687)

=== CONCISE 설정으로 평가 중 ===


...


CONCISE 설정 결과:
  ROUGE1: 0.2822 (±0.3867)
  ROUGE2: 0.1654 (±0.3290)
  ROUGEL: 0.2716 (±0.3798)

=== FOCUSED 설정으로 평가 중 ===


focused 요약 생성: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]



FOCUSED 설정 결과:
  ROUGE1: 0.2987 (±0.3944)
  ROUGE2: 0.1857 (±0.3481)
  ROUGEL: 0.2957 (±0.3937)

=== BALANCED 설정으로 평가 중 ===


balanced 요약 생성: 100%|██████████| 100/100 [01:11<00:00,  1.40it/s]


BALANCED 설정 결과:
  ROUGE1: 0.3047 (±0.3865)
  ROUGE2: 0.1767 (±0.3322)
  ROUGEL: 0.2931 (±0.3796)

파라미터 설정별 성능 비교
Config       ROUGE-1      ROUGE-2      ROUGE-L     
----------------------------------------------------------------------
default      0.3131       0.1828       0.3025      
concise      0.2822       0.1654       0.2716      
focused      0.2987       0.1857       0.2957      
balanced     0.3047       0.1767       0.2931      

최고 성능 설정:
  ROUGE1: default (0.3131)
  ROUGE2: focused (0.1857)
  ROUGEL: default (0.3025)

샘플 1 비교
참조 요약: ‘이집트 다음은 중국?’ 온라인 《월 스트리트 저널》은 1월 31일 중국관련 기사들의 목록을 올리면서 그런 제목을 달았다. 전날 온라인 《포브스》가 〈튀니지 다음은 이집트, 이집트 다음은?〉이라는 제목을 단 고든 창(Gordon G. Chang)의 칼럼을 게재한 데 대한 답이었다.
----------------------------------------------------------------------
DEFAULT 설정:
  온라인 《월 스트리트 저널》은 1월 31일 중국관련 기사들의 목록을 올리면서 '베이징, (이집트의) 항의시위 보도를 차단'이라는 기사를 1월 31일자에 실었는데, 이 기사는 중국 당국이 트위터와 비슷한 마이크로 블로그(웨이보·微博)의 검색창에서 이집트(아이지·埃及)라는 단어를 차단했다고 전하고 “중국 지도자들이 이집트에서와 유사한 정치개혁 요구가 중국에서도 제기될

# 키워드 추출

In [25]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran
from keybert import KeyBERT
import networkx as nx
from collections import Counter
import re
from tqdm import tqdm
import warnings
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
warnings.filterwarnings('ignore')

In [26]:
class KeywordExtractorComparison:
    def __init__(self):
        """키워드 추출 모델들 초기화"""
        print("모델들 초기화 중...")

        # Komoran 형태소 분석기
        self.komoran = Komoran()

        # KeyBERT 모델
        self.keybert = KeyBERT('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

        # TF-IDF 벡터라이저 (나중에 fit)
        self.tfidf_vectorizer = None

        print("모델 초기화 완료!")

    def _noun_tokenizer(self, text):
        """명사 추출 토크나이저"""
        try:
            nouns = self.komoran.nouns(text)
            return [noun for noun in nouns if len(noun) >= 2]
        except:
            return []

    def _preprocess_text(self, text):
        """텍스트 전처리"""
        if not text:
            return ""
        # 특수문자 제거, 공백 정리
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def extract_keybert(self, text, top_k=10):
        """KeyBERT로 키워드 추출 - 수정된 버전"""
        try:
            if not text.strip():
                return []

            # KeyBERT 버전에 따른 파라미터 조정
            try:
                # 최신 버전 시도
                keywords = self.keybert.extract_keywords(
                    text,
                    keyphrase_ngram_range=(1, 2),
                    stop_words=['하는', '있는', '위한', '통한', '되지', '하고', '것은', '것이', '수는', '때는'],
                    top_n=top_k,  # top_k -> top_n
                    use_mmr=True,
                    diversity=0.5
                )
            except TypeError:
                # 구버전 호환성
                keywords = self.keybert.extract_keywords(
                    text,
                    keyphrase_ngram_range=(1, 2),
                    stop_words=['하는', '있는', '위한', '통한', '되지', '하고', '것은', '것이', '수는', '때는'],
                    top_k=top_k,
                    use_mmr=True,
                    diversity=0.5
                )
            except:
                # 기본 버전
                keywords = self.keybert.extract_keywords(
                    text,
                    keyphrase_ngram_range=(1, 2),
                    stop_words=['하는', '있는', '위한', '통한', '되지', '하고', '것은', '것이', '수는', '때는'],
                    top_n=top_k
                )

            return [kw[0] for kw in keywords]

        except Exception as e:
            print(f"KeyBERT 오류: {e}")
            # KeyBERT 실패시 간단한 대체 방법
            try:
                nouns = self.komoran.nouns(text)
                noun_counts = Counter([noun for noun in nouns if len(noun) >= 2])
                return [word for word, count in noun_counts.most_common(top_k)]
            except:
                return []

    def fit_tfidf(self, texts):
        """TF-IDF 벡터라이저 학습"""
        print("TF-IDF 벡터라이저 학습 중...")
        try:
            self.tfidf_vectorizer = TfidfVectorizer(
                tokenizer=self._noun_tokenizer,
                ngram_range=(1, 2),
                max_features=5000,
                min_df=2,
                max_df=0.95
            )
            self.tfidf_vectorizer.fit(texts)
            print("TF-IDF 학습 완료!")
        except Exception as e:
            print(f"TF-IDF 학습 오류: {e}")
            # 간단한 대체 벡터라이저
            self.tfidf_vectorizer = TfidfVectorizer(
                ngram_range=(1, 1),
                max_features=1000,
                min_df=1
            )
            self.tfidf_vectorizer.fit(texts)

    def extract_tfidf(self, text, top_k=10):
        """TF-IDF로 키워드 추출"""
        try:
            if not text.strip() or self.tfidf_vectorizer is None:
                return []

            # 텍스트의 TF-IDF 점수
            tfidf_vector = self.tfidf_vectorizer.transform([text])
            feature_names = self.tfidf_vectorizer.get_feature_names_out()
            scores = tfidf_vector.toarray()[0]

            # 상위 키워드 추출
            top_indices = scores.argsort()[-top_k:][::-1]
            keywords = [feature_names[i] for i in top_indices if scores[i] > 0]

            return keywords
        except Exception as e:
            print(f"TF-IDF 오류: {e}")
            return []

    def extract_textrank(self, text, top_k=10):
        """TextRank로 키워드 추출"""
        try:
            if not text.strip():
                return []

            # 명사 추출
            nouns = self.komoran.nouns(text)
            nouns = [noun for noun in nouns if len(noun) >= 2]

            if len(nouns) < 3:
                # TextRank 실패시 빈도 기반 대체
                noun_counts = Counter(nouns)
                return [word for word, count in noun_counts.most_common(top_k)]

            # 동시 출현 그래프 생성
            graph = nx.Graph()

            # 윈도우 크기 5로 동시 출현 관계 생성
            window_size = 5
            for i in range(len(nouns) - window_size + 1):
                window = nouns[i:i + window_size]
                for j in range(len(window)):
                    for k in range(j + 1, len(window)):
                        if window[j] != window[k]:
                            if graph.has_edge(window[j], window[k]):
                                graph[window[j]][window[k]]['weight'] += 1
                            else:
                                graph.add_edge(window[j], window[k], weight=1)

            # PageRank 계산
            if len(graph.nodes()) == 0:
                noun_counts = Counter(nouns)
                return [word for word, count in noun_counts.most_common(top_k)]

            pagerank_scores = nx.pagerank(graph, weight='weight')

            # 상위 키워드 반환
            sorted_keywords = sorted(pagerank_scores.items(),
                                   key=lambda x: x[1], reverse=True)

            return [kw[0] for kw in sorted_keywords[:top_k]]
        except Exception as e:
            print(f"TextRank 오류: {e}")
            # 대체 방법: 빈도 기반
            try:
                nouns = self.komoran.nouns(text)
                noun_counts = Counter([noun for noun in nouns if len(noun) >= 2])
                return [word for word, count in noun_counts.most_common(top_k)]
            except:
                return []

In [27]:
def load_kptimes_data(num_samples=100):
    """KPTimes 데이터셋 로드 및 전처리"""
    print("KPTimes 데이터셋 로딩 중...")

    try:
        # 데이터셋 로드
        dataset = load_dataset("taln-ls2n/kptimes")
        test_data = dataset['test']

        print(f"전체 테스트 데이터: {len(test_data)}개")

        # 랜덤 샘플링
        if num_samples < len(test_data):
            indices = random.sample(range(len(test_data)), num_samples)
            sampled_data = [test_data[i] for i in indices]
        else:
            sampled_data = list(test_data)

        # 데이터 전처리
        processed_data = []
        for item in sampled_data:
            # title + abstract를 합쳐서 전체 텍스트 생성
            title = item.get('title', '')
            abstract = item.get('abstract', '')
            full_text = f"{title} {abstract}".strip()

            # keyphrases 처리 (리스트 형태로 변환)
            keyphrases = item.get('keyphrases', [])
            if isinstance(keyphrases, str):
                # 문자열인 경우 분할
                keyphrases = [kp.strip() for kp in keyphrases.split(',') if kp.strip()]
            elif not isinstance(keyphrases, list):
                keyphrases = []

            if full_text and keyphrases:
                processed_data.append({
                    'text': full_text,
                    'keywords': keyphrases
                })

        print(f"처리된 데이터: {len(processed_data)}개")
        return processed_data

    except Exception as e:
        print(f"데이터 로드 오류: {e}")
        return []

In [28]:
def calculate_metrics(predicted_keywords, true_keywords, k=5):
    """평가 지표 계산"""
    # 상위 k개만 고려
    pred_k = set([kw.lower().strip() for kw in predicted_keywords[:k] if kw])
    true_set = set([kw.lower().strip() for kw in true_keywords if kw])

    if len(true_set) == 0 or len(pred_k) == 0:
        return {"precision": 0, "recall": 0, "f1": 0}

    # 교집합
    intersection = pred_k.intersection(true_set)

    # Precision@K
    precision = len(intersection) / len(pred_k) if len(pred_k) > 0 else 0

    # Recall@K
    recall = len(intersection) / len(true_set)

    # F1@K
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [29]:
def evaluate_extractors(num_samples=100):
    """키워드 추출기들 성능 평가"""
    extractor = KeywordExtractorComparison()

    # 데이터 로드
    processed_data = load_kptimes_data(num_samples)

    if len(processed_data) == 0:
        print("처리할 데이터가 없습니다.")
        return None, None

    # 전체 텍스트 수집 (TF-IDF 학습용)
    all_texts = [item['text'] for item in processed_data]

    # TF-IDF 벡터라이저 학습
    extractor.fit_tfidf(all_texts)

    print(f"\n{len(processed_data)}개 샘플로 평가를 시작합니다...")

    results = {
        "KeyBERT": {"precision": [], "recall": [], "f1": []},
        "TF-IDF": {"precision": [], "recall": [], "f1": []},
        "TextRank": {"precision": [], "recall": [], "f1": []}
    }

    sample_results = []

    for i, item in enumerate(tqdm(processed_data, desc="키워드 추출 평가")):
        text = item['text']
        true_keywords = item['keywords']

        # 전처리
        clean_text = extractor._preprocess_text(text)

        # 각 방법으로 키워드 추출
        keybert_kw = extractor.extract_keybert(clean_text, top_k=10)
        tfidf_kw = extractor.extract_tfidf(clean_text, top_k=10)
        textrank_kw = extractor.extract_textrank(clean_text, top_k=10)

        # 평가 지표 계산 (상위 5개 기준)
        keybert_metrics = calculate_metrics(keybert_kw, true_keywords, k=5)
        tfidf_metrics = calculate_metrics(tfidf_kw, true_keywords, k=5)
        textrank_metrics = calculate_metrics(textrank_kw, true_keywords, k=5)

        # 결과 저장
        for method, metrics in [("KeyBERT", keybert_metrics),
                               ("TF-IDF", tfidf_metrics),
                               ("TextRank", textrank_metrics)]:
            results[method]["precision"].append(metrics["precision"])
            results[method]["recall"].append(metrics["recall"])
            results[method]["f1"].append(metrics["f1"])

        # 샘플 결과 저장 (처음 3개)
        if i < 3:
            sample_results.append({
                "text": text[:150] + "..." if len(text) > 150 else text,
                "true_keywords": true_keywords[:10],  # 너무 많으면 일부만
                "keybert": keybert_kw[:5],
                "tfidf": tfidf_kw[:5],
                "textrank": textrank_kw[:5]
            })

    return results, sample_results

In [30]:
def print_evaluation_results(results):
    """평가 결과 출력"""
    print("\n" + "="*80)
    print("키워드 추출 모델 성능 비교 (KPTimes 데이터셋)")
    print("="*80)

    methods = ["KeyBERT", "TF-IDF", "TextRank"]

    # 헤더
    print(f"{'Method':<12} {'Precision@5':<18} {'Recall@5':<18} {'F1@5':<18}")
    print("-" * 80)

    best_scores = {"precision": 0, "recall": 0, "f1": 0}
    best_method = {"precision": "", "recall": "", "f1": ""}

    for method in methods:
        prec_mean = np.mean(results[method]["precision"])
        prec_std = np.std(results[method]["precision"])

        rec_mean = np.mean(results[method]["recall"])
        rec_std = np.std(results[method]["recall"])

        f1_mean = np.mean(results[method]["f1"])
        f1_std = np.std(results[method]["f1"])

        print(f"{method:<12} {prec_mean:.3f} (±{prec_std:.3f}){'':<4} "
              f"{rec_mean:.3f} (±{rec_std:.3f}){'':<4} "
              f"{f1_mean:.3f} (±{f1_std:.3f})")

        # 최고 성능 추적
        if prec_mean > best_scores["precision"]:
            best_scores["precision"] = prec_mean
            best_method["precision"] = method
        if rec_mean > best_scores["recall"]:
            best_scores["recall"] = rec_mean
            best_method["recall"] = method
        if f1_mean > best_scores["f1"]:
            best_scores["f1"] = f1_mean
            best_method["f1"] = method

    print("\n" + "="*80)
    print("최고 성능:")
    print(f"  Precision@5: {best_method['precision']} ({best_scores['precision']:.3f})")
    print(f"  Recall@5: {best_method['recall']} ({best_scores['recall']:.3f})")
    print(f"  F1@5: {best_method['f1']} ({best_scores['f1']:.3f})")
    print("="*80)

In [31]:
def print_sample_results(sample_results):
    """샘플 결과 출력"""
    print("\n" + "="*80)
    print("샘플 키워드 추출 결과")
    print("="*80)

    for i, sample in enumerate(sample_results):
        print(f"\n--- 샘플 {i+1} ---")
        print(f"텍스트: {sample['text']}")
        print(f"정답 키워드: {sample['true_keywords']}")
        print(f"KeyBERT: {sample['keybert']}")
        print(f"TF-IDF: {sample['tfidf']}")
        print(f"TextRank: {sample['textrank']}")
        print("-" * 60)

In [32]:
def main():
    """메인 실행 함수"""
    print("KPTimes 데이터셋을 사용한 키워드 추출 성능 평가")
    print("="*80)

    # 성능 평가 (100개 샘플로 시작, 필요시 조정 가능)
    results, sample_results = evaluate_extractors(num_samples=100)

    if results is None:
        print("평가를 완료할 수 없습니다.")
        return

    # 결과 출력
    print_evaluation_results(results)
    print_sample_results(sample_results)

    print("\n평가 완료!")
    print("* Precision@5: 상위 5개 키워드 중 정답 비율")
    print("* Recall@5: 전체 정답 키워드 중 상위 5개에서 찾은 비율")
    print("* F1@5: Precision과 Recall의 조화평균")
    print("* 점수가 높을수록 좋은 성능을 의미합니다.")

In [33]:
main()

KPTimes 데이터셋을 사용한 키워드 추출 성능 평가
모델들 초기화 중...
모델 초기화 완료!
KPTimes 데이터셋 로딩 중...
전체 테스트 데이터: 20000개
처리된 데이터: 100개
TF-IDF 벡터라이저 학습 중...
TF-IDF 학습 완료!

100개 샘플로 평가를 시작합니다...


키워드 추출 평가: 100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


키워드 추출 모델 성능 비교 (KPTimes 데이터셋)
Method       Precision@5        Recall@5           F1@5              
--------------------------------------------------------------------------------
KeyBERT      0.020 (±0.060)     0.021 (±0.067)     0.020 (±0.060)
TF-IDF       0.000 (±0.000)     0.000 (±0.000)     0.000 (±0.000)
TextRank     0.000 (±0.000)     0.000 (±0.000)     0.000 (±0.000)

최고 성능:
  Precision@5: KeyBERT (0.020)
  Recall@5: KeyBERT (0.021)
  F1@5: KeyBERT (0.020)

샘플 키워드 추출 결과

--- 샘플 1 ---
텍스트: Syphilis infections in Japan top 6,000 for first time since 1970  Cases of syphilis infections this year totaled 6,096 as of Nov. 18, exceeding an ann...
정답 키워드: ['syphilis', 'disease', 'drugs', 'sex', 'medicine']
KeyBERT: ['syphilis patients', 'totaled 096', 'japan top', '1970 cases', 'called treponema']
TF-IDF: []
TextRank: []
------------------------------------------------------------

--- 샘플 2 ---
텍스트: U.N.'s new rights chief Michelle Bachelet presses for new body on crimes against Mya

In [14]:
data = load_dataset("taln-ls2n/kptimes")

README.md:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

kptimes.py:   0%|          | 0.00/7.79k [00:00<?, ?B/s]

The repository for taln-ls2n/kptimes contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/taln-ls2n/kptimes.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


test.jsonl:   0%|          | 0.00/84.7M [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/50.9M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu', 'date', 'categories'],
        num_rows: 259923
    })
    test: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu', 'date', 'categories'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu', 'date', 'categories'],
        num_rows: 10000
    })
})